## RL for Predictive Maintenance

- Using SIT data
- Tested with Case-1, 200 records

In [ ]:
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
import matplotlib.pyplot as plt
import os
from typing import Tuple, Dict, Any, List

### Global settings

In [ ]:
# --- Global Parameters ---
# These can be easily modified for experimentation.

# Data and Environment settings
IOT_FILE = 'SIT_84'
DATA_FILE = f'{IOT_FILE}.csv' # 'data\Case_1_1K_Data.csv'
TEST_FILE = f'SIT_10.csv' # 'data\Case_1_Test.csv'
WEAR_THRESHOLD = 290.0  # Threshold for tool wear before it's considered failed

# Reward structure
R1_CONTINUE = 2.0      # Reward for continuing with a healthy tool
R2_REPLACE = -5.0      # Reward for a timely replacement
R3_VIOLATION = -30.0   # Penalty for violating the wear threshold

# Training settings
EPISODES = 1000          # Total number of episodes to train each agent
LEARNING_RATE = 5e-3    # Learning rate for optimizers
GAMMA = 0.99            # Discount factor for future rewards

# Plotting settings
SMOOTH_WINDOW = 10     # Window size for moving average smoothing on plots

# Model saving
SAVE_MODEL = True
MODEL_FILE = f'models/{IOT_FILE}_Model.h5'

### Plotting

In [ ]:
# --- Task 4: Plotting the Comparison ---

def smooth(data: List[float], window_size: int) -> np.ndarray:
    """
    Applies a simple moving average to a list of data.
    Handles NaNs by ignoring them in the window calculation.
    """
    return pd.Series(data).rolling(window=window_size, min_periods=1).mean().to_numpy()

def plot_metrics(reinforce_metrics: Dict, ppo_metrics: Dict, window: int, mode: str = "COMBINED"):
    """
    Generates performance metric plots either combined or separate.
    
    Args:
        reinforce_metrics (Dict): Dictionary containing REINFORCE metrics
        ppo_metrics (Dict): Dictionary containing PPO metrics
        window (int): Window size for smoothing the curves
        mode (str): Either "COMBINED" or "SEPARATE" for plotting style
    """
    W, H = 16, 6
    FONTSIZE_SUPER = 18
    FONTSIZE_TITLE = 12
    FONTSIZE_LABEL = 10
    FONTSIZE_TICK = 9
    BACKGROUND_COLOR = '#f8f8f8'  # Lighter grey background
    
    def setup_subplot(ax, title, xlabel, ylabel):
        """Helper function to set up common subplot properties"""
        ax.set_title(title, fontsize=FONTSIZE_TITLE)
        ax.set_xlabel(xlabel, fontsize=FONTSIZE_LABEL)
        ax.set_ylabel(ylabel, fontsize=FONTSIZE_LABEL)
        ax.tick_params(labelsize=FONTSIZE_TICK)
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.set_facecolor(BACKGROUND_COLOR)
    
    # Get data ranges for consistent y-axis limits
    def get_data_ranges(metrics_config):
        ranges = {}
        for config in metrics_config:
            key = config['data']
            if key == 'margins':
                r_data = pd.Series(reinforce_metrics[key]).rolling(window, min_periods=1).mean()
                p_data = pd.Series(ppo_metrics[key]).rolling(window, min_periods=1).mean()
            else:
                r_data = smooth(reinforce_metrics[key], window)
                p_data = smooth(ppo_metrics[key], window)
            
            all_data = np.concatenate([r_data, p_data])
            ranges[key] = (np.nanmin(all_data), np.nanmax(all_data))
        return ranges
        
    if mode.upper() == "COMBINED":
        # Create single figure with both algorithms
        fig, axs = plt.subplots(2, 2, figsize=(W, H))
        fig.suptitle('REINFORCE vs. PPO Performance for Predictive Maintenance', fontsize=FONTSIZE_SUPER)
        
        # Plot all metrics with both algorithms
        metrics_config = [
            {'data': 'rewards', 'title': 'Learning Curve (Smoothed Average Reward)', 
             'ylabel': 'Average Reward'},
            {'data': 'replacements', 'title': 'Replacements per episode (Smoothed)', 
             'ylabel': 'Replacement Rate'},
            {'data': 'violations', 'title': 'Threshold Violations per episode (Smoothed)', 
             'ylabel': 'Violation Rate'},
            {'data': 'margins', 'title': 'Wear Margin Before Replacement (Smoothed)', 
             'ylabel': 'Wear Margin'}
        ]
        
        for idx, config in enumerate(metrics_config):
            ax = axs[idx // 2, idx % 2]
            if config['data'] == 'margins':
                r_data = pd.Series(reinforce_metrics[config['data']]).rolling(window, min_periods=1).mean()
                p_data = pd.Series(ppo_metrics[config['data']]).rolling(window, min_periods=1).mean()
            else:
                r_data = smooth(reinforce_metrics[config['data']], window)
                p_data = smooth(ppo_metrics[config['data']], window)
                
            ax.plot(r_data, label='REINFORCE', alpha=0.6)
            ax.plot(p_data, label='PPO', alpha=0.6)
            setup_subplot(ax, config['title'], 'Episode', config['ylabel'])
            ax.legend()
        
    else:  # SEPARATE mode
        # First calculate data ranges for consistent scaling
        metrics_config = [
            {'data': 'rewards', 'color': 'blue', 'title': 'Learning Curve (Smoothed Average Reward)', 
             'ylabel': 'Average Reward'},
            {'data': 'replacements', 'color': 'green', 'title': 'Replacements per episode (Smoothed)', 
             'ylabel': 'Replacement Rate'},
            {'data': 'violations', 'color': 'red', 'title': 'Threshold Violations per episode (Smoothed)', 
             'ylabel': 'Violation Rate'},
            {'data': 'margins', 'color': 'purple', 'title': 'Wear Margin Before Replacement (Smoothed)', 
             'ylabel': 'Wear Margin'}
        ]
        data_ranges = get_data_ranges(metrics_config)
        
        # Create separate figures for each algorithm
        for algo, metrics in [("REINFORCE", reinforce_metrics), ("PPO", ppo_metrics)]:
            fig, axs = plt.subplots(2, 2, figsize=(W, H))
            fig.patch.set_facecolor(BACKGROUND_COLOR)  # Set figure background
            fig.suptitle(f'{algo} Performance Metrics for Predictive Maintenance', fontsize=FONTSIZE_SUPER)
            
            for idx, config in enumerate(metrics_config):
                ax = axs[idx // 2, idx % 2]
                if config['data'] == 'margins':
                    data = pd.Series(metrics[config['data']]).rolling(window, min_periods=1).mean()
                else:
                    data = smooth(metrics[config['data']], window)
                    
                ax.plot(data, color=config['color'], alpha=0.6)
                setup_subplot(ax, config['title'], 'Episode', config['ylabel'])
                
                # Set consistent y-axis limits
                ymin, ymax = data_ranges[config['data']]
                padding = (ymax - ymin) * 0.1  # Add 10% padding
                # ax.set_ylim(ymin - padding, ymax + padding)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

### Custom RL Environment for milling tool maintenance
- OpenAI's Gymnasium open standards format

In [ ]:
# --- Task 1: Custom Gymnasium Environment ---
# We define the environment for the milling tool maintenance problem.

class MT_Env(gym.Env):
    """
    Custom Gymnasium environment for Milling Tool Predictive Maintenance.

    The agent observes the current tool wear and decides whether to continue
    using the tool or replace it.

    Observation Space: A single continuous value representing the current tool wear.
    Action Space: Two discrete actions - 0 (CONTINUE) or 1 (REPLACE_TOOL).
    """
    metadata = {'render_modes': ['human']}

    def __init__(self, data_file: str, wear_threshold: float, r1: float, r2: float, r3: float):
        """
        Initializes the environment.

        Args:
            data_file (str): Path to the CSV file with sensor data and tool wear.
            wear_threshold (float): The maximum allowable tool wear.
            r1 (float): Reward for continuing.
            r2 (float): Reward for a timely replacement.
            r3 (float): Penalty for a threshold violation.
        """
        super(MT_Env, self).__init__()

        # Load and preprocess data
        if not os.path.exists(data_file):
            raise FileNotFoundError(f"Data file not found at {data_file}. Please ensure the file is in the correct directory.")
        self.df = pd.read_csv(data_file)
        self.max_steps = len(self.df) - 1

        # Environment parameters
        self.wear_threshold = wear_threshold
        self.r1_continue = r1
        self.r2_replace = r2
        self.r3_violation = r3

        # Define action and observation spaces
        self.action_space = spaces.Discrete(2)  # 0: CONTINUE, 1: REPLACE_TOOL
        # The observation is the current tool wear, normalized for better learning
        self.observation_space = spaces.Box(low=0, high=self.wear_threshold * 1.5, shape=(1,), dtype=np.float32)

        # Internal state
        self.current_step = 0

    def reset(self, seed=None, options=None) -> Tuple[np.ndarray, Dict]:
        """
        Resets the environment to its initial state for a new episode.
        """
        super().reset(seed=seed)
        self.current_step = 0
        initial_observation = self._get_observation()
        return initial_observation, {}

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, Dict]:
        """
        Executes one time step within the environment.
        """
        terminated = False
        truncated = False
        reward = 0
        info = {'violation': False, 'replacement': False, 'margin': np.nan}

        current_wear = self.df.loc[self.current_step, 'tool_wear']

        if action == 1:  # Action: REPLACE_TOOL
            reward = self.r2_replace
            terminated = True
            info['replacement'] = True
            info['margin'] = self.wear_threshold - current_wear
        elif action == 0:  # Action: CONTINUE
            self.current_step += 1
            if self.current_step > self.max_steps:
                # Reached the end of data without replacement or violation
                truncated = True
                reward = 0 # No reward or penalty if data runs out
            else:
                next_wear = self.df.loc[self.current_step, 'tool_wear']
                if next_wear >= self.wear_threshold:
                    reward = self.r3_violation
                    terminated = True
                    info['violation'] = True
                    # $$$ Avoid NANs for margin. Even if NOT replaced, see what the margin is
                    info['margin'] = self.wear_threshold - current_wear
                else:
                    reward = self.r1_continue
        
        observation = self._get_observation()
        
        return observation, reward, terminated, truncated, info

    def _get_observation(self) -> np.ndarray:
        """
        Gets the current observation from the dataframe.
        """
        wear = self.df.loc[self.current_step, 'tool_wear']
        return np.array([wear], dtype=np.float32)

    def render(self, mode='human'):
        """
        Renders the environment state (optional, for visualization).
        """
        wear = self.df.loc[self.current_step, 'tool_wear']
        print(f"Step: {self.current_step}, Tool Wear: {wear:.2f}")


### REINFORCE Algorithm Implementation
- Custom implementation of the REINFORCE algorithm
- Employs OpenAI's Stable Baselines3 open-source standards for wider research adoption

In [ ]:
# --- Task 2: REINFORCE Algorithm Implementation ---
# A custom implementation of the REINFORCE algorithm with a structure
# similar to Stable Baselines3 for easy comparison.
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0)
        return self.net(x)

class REINFORCE:
    """
    A custom REINFORCE agent.
    """
    def __init__(self, policy, env, learning_rate=0.001, gamma=0.99, model_file=None):
        self.env = env
        self.policy = policy
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.model_name = "REINFORCE"
        self.model_file = model_file

    def predict(self, obs: np.ndarray) -> int:
        """
        Selects an action based on the current observation.
        Uses the same action selection logic as training for consistency.
        """
        with torch.no_grad():  # Important to disable gradients for prediction
            state_tensor = torch.FloatTensor(obs).unsqueeze(0)
            probs = self.policy(state_tensor)
            dist = Categorical(probs)
            action = dist.sample()  # Sample action like in training
            return action.item()        

    def learn(self, total_episodes: int) -> Dict[str, List]:
        """
        Trains the agent for a given number of episodes.
        """
        print(f"--- Training {self.model_name} ---")
        # Metrics collectors
        all_rewards = []
        all_violations = []
        all_replacements = []
        all_margins = []

        for episode in range(total_episodes):
            log_probs = []
            rewards = []
            obs, _ = self.env.reset()
            done = False
            
            episode_info = {'violation': 0, 'replacement': 0, 'margin': np.nan}

            while not done:
                state_tensor = torch.FloatTensor(obs).unsqueeze(0)
                probs = self.policy(state_tensor)
                dist = Categorical(probs)
                action = dist.sample()

                log_prob = dist.log_prob(action)
                log_probs.append(log_prob)

                obs, reward, terminated, truncated, info = self.env.step(action.item())
                rewards.append(reward)
                done = terminated or truncated

                if info.get('violation'): episode_info['violation'] = 1
                if info.get('replacement'): episode_info['replacement'] = 1
                if not np.isnan(info.get('margin')): episode_info['margin'] = info.get('margin')
            
            # Collect metrics for this episode
            all_rewards.append(sum(rewards))
            all_violations.append(episode_info['violation'])
            all_replacements.append(episode_info['replacement'])
            all_margins.append(episode_info['margin'])

            # Calculate discounted returns
            returns = []
            G = 0
            for r in reversed(rewards):
                G = r + self.gamma * G
                returns.insert(0, G)
            returns = torch.tensor(returns, dtype=torch.float32)
            
            # Normalize returns for stability, but only if there's more than one step
            # to avoid std() of a single item becoming NaN.
            if len(returns) > 1:
                returns = (returns - returns.mean()) / (returns.std() + 1e-9)

            # Calculate policy loss and update weights
            policy_loss = []
            for log_prob, G in zip(log_probs, returns):
                policy_loss.append(-log_prob * G)
            
            self.optimizer.zero_grad()
            policy_loss = torch.cat(policy_loss).sum()
            policy_loss.backward()
            self.optimizer.step()

            if (episode + 1) % 100 == 0:
                print(f"Episode {episode + 1}/{total_episodes}, Total Reward: {sum(rewards):.2f}")
        
        print("--- Training Complete ---")
        
        # Save model if file path was provided
        if self.model_file is not None:
            try:
                # Create directory if it doesn't exist
                os.makedirs(os.path.dirname(self.model_file), exist_ok=True)
                # Save the model
                torch.save({
                    'policy_state_dict': self.policy.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'input_dim': self.policy.net[0].in_features,
                    'output_dim': self.policy.net[-2].out_features
                }, self.model_file)
                print(f"Model saved successfully to: {self.model_file}")
            except Exception as e:
                print(f"Error saving model: {str(e)}")

        return {
            "rewards": all_rewards,
            "violations": all_violations,
            "replacements": all_replacements,
            "margins": all_margins
        }
    
    

# --- Task 3: Comparison with Stable Baselines3 PPO ---
# We use a callback to collect metrics during PPO training to ensure a
# fair, episode-by-episode comparison with our REINFORCE agent.

class MetricsCallback(BaseCallback):
    """
    A custom callback for saving metrics at the end of each episode.
    """
    def __init__(self, verbose=0):
        super(MetricsCallback, self).__init__(verbose)
        self.rewards = []
        self.violations = []
        self.replacements = []
        self.margins = []

    def _on_step(self) -> bool:
        # Check if an episode has ended
        if self.locals['dones'][0]:
            # The Monitor wrapper ensures the info dict has 'episode' stats
            info = self.locals['infos'][0]
            self.rewards.append(info['episode']['r'])
            self.violations.append(1 if info.get('violation') else 0)
            self.replacements.append(1 if info.get('replacement') else 0)
            self.margins.append(info.get('margin', np.nan))
        return True

def train_ppo(env: gym.Env, total_episodes: int) -> Dict[str, List]:
    """
    Trains a PPO agent and collects metrics using a callback.
    """
    print("--- Training PPO ---")
    callback = MetricsCallback()
    
    # Since we train by episode, we set a very high number of timesteps
    # and rely on the custom loop to stop at the right number of episodes.
    total_timesteps = 1_000_000 

    model = PPO("MlpPolicy", env, verbose=0, learning_rate=LEARNING_RATE, gamma=GAMMA)
    
    # We can now use the standard model.learn() with our callback,
    # but we need a way to stop after N episodes. A custom loop is still clearer.
    obs, _ = env.reset()
    ep_count = 0
    while ep_count < total_episodes:
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, terminated, truncated, info = env.step(action)
        
        # Manually trigger the callback's on_step method
        callback.model = model
        callback.locals = {'dones': [terminated or truncated], 'infos': [info]}
        callback._on_step()
        
        if terminated or truncated:
            ep_count += 1
            if (ep_count) % 100 == 0:
                print(f"Episode {ep_count}/{total_episodes}")
            obs, _ = env.reset()

    print("--- Training Complete ---")
    return {
        "rewards": callback.rewards,
        "violations": callback.violations,
        "replacements": callback.replacements,
        "margins": callback.margins
    }


### Test saved RL agent

In [ ]:
def test_saved_model(model_file: str, test_data_file: str, wear_threshold: float, r1: float, r2: float, r3: float):
    """
    Tests a saved model on new sensor data and visualizes the results.
    
    Args:
        model_file (str): Path to the saved model file
        test_data_file (str): Path to the new sensor data file
        wear_threshold (float): Threshold for tool wear
        r1, r2, r3 (float): Reward parameters for the environment
    """
    # Create test environment
    test_env = MT_Env(
        data_file=test_data_file,
        wear_threshold=wear_threshold,
        r1=r1, r2=r2, r3=r3
    )
    
    # Load the model
    policy = PolicyNetwork(
        input_dim=test_env.observation_space.shape[0],
        output_dim=test_env.action_space.n
    )
    agent = REINFORCE(policy, test_env)
    
    try:
        checkpoint = torch.load(model_file)
        agent.policy.load_state_dict(checkpoint['policy_state_dict'])
        print(f"Model loaded successfully from: {model_file}")
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return
    
    # Test metrics
    total_steps = 0
    total_replacements = 0
    total_violations = 0
    margins = []
    replacement_points = []
    violation_points = []
    wear_trajectory = []
    
    # Run test episode
    obs, _ = test_env.reset()
    done = False
    step = 0
    
    while not done:
        action = agent.predict(obs)
        obs, reward, terminated, truncated, info = test_env.step(action)
        wear = obs[0]
        wear_trajectory.append(wear)
        
        if info.get('replacement'):
            total_replacements += 1
            replacement_points.append((step, wear))
            margins.append(wear_threshold - wear)
        
        if info.get('violation'):
            total_violations += 1
            violation_points.append((step, wear))
            margins.append(wear_threshold - wear)
        
        step += 1
        done = terminated or truncated
    
    total_steps = step
    
    # Calculate metrics
    metrics = {
        'total_steps': total_steps,
        'replacement_rate': total_replacements / total_steps,
        'violation_rate': total_violations / total_steps,
        'avg_margin': np.mean(margins) if margins else np.nan,
        'min_margin': np.min(margins) if margins else np.nan
    }
    
    # Plotting
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))
    fig.suptitle('Model Test Results', fontsize=11)
    
    # Plot 1: Wear trajectory with events
    ax1.plot(wear_trajectory, 'b-', alpha=0.6, label='Tool Wear')
    ax1.axhline(y=wear_threshold, color='r', linestyle='--', alpha=0.6, label='Wear Threshold')
    
    # Plot replacement and violation points
    if replacement_points:
        points = np.array(replacement_points)
        ax1.scatter(points[:, 0], points[:, 1], color='g', marker='^', 
                label='Replacements', alpha=0.6)
    if violation_points:
        points = np.array(violation_points)
        ax1.scatter(points[:, 0], points[:, 1], color='r', marker='x', 
                label='Violations', alpha=0.6)
    
    ax1.set_title('Tool Wear Trajectory')
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Wear')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Plot 2: Margins histogram
    if margins:
        ax2.hist(margins, bins=20, alpha=0.6, color='purple')
        ax2.axvline(x=0, color='r', linestyle='--', alpha=0.6)
        ax2.set_title('Distribution of Margins at Decision Points')
        ax2.set_xlabel('Margin (Threshold - Wear)')
        ax2.set_ylabel('Frequency')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    print("\nTest Results:")
    print(f"Total Steps: {metrics['total_steps']}")
    print(f"Replacement Rate: {metrics['replacement_rate']:.4f}")
    print(f"Violation Rate: {metrics['violation_rate']:.4f}")
    print(f"Average Margin: {metrics['avg_margin']:.2f}")
    print(f"Minimum Margin: {metrics['min_margin']:.2f}")
    
    return metrics

### Training

In [ ]:
# Create the environment
try:
    # Create a base environment for the custom REINFORCE agent
    reinforce_env = MT_Env(
        data_file=DATA_FILE,
        wear_threshold=WEAR_THRESHOLD,
        r1=R1_CONTINUE,
        r2=R2_REPLACE,
        r3=R3_VIOLATION
    )
    # It's good practice to check if the custom environment is valid
    check_env(reinforce_env)
    print("Environment created and checked successfully.")

    # Create a separate, monitored environment for the Stable Baselines3 PPO agent
    ppo_env = Monitor(MT_Env(
        data_file=DATA_FILE,
        wear_threshold=WEAR_THRESHOLD,
        r1=R1_CONTINUE,
        r2=R2_REPLACE,
        r3=R3_VIOLATION
    ))

except FileNotFoundError as e:
    print(e)
    # Exit if the data file is not found
    exit()

# Train REINFORCE agent
print('\n\n Training REINFORCE agent...')
reinforce_policy = PolicyNetwork(reinforce_env.observation_space.shape[0], reinforce_env.action_space.n)
reinforce_agent = REINFORCE(reinforce_policy, reinforce_env, learning_rate=LEARNING_RATE, gamma=GAMMA, model_file=MODEL_FILE)
reinforce_metrics = reinforce_agent.learn(total_episodes=EPISODES)
print('\n -- REINFORCE agent training complete.')

# Train PPO agent
print('\n\n Training PPO agent...')
ppo_metrics = train_ppo(ppo_env, total_episodes=EPISODES)
print('\n -- PPO agent training complete.')

### Results plots 

In [ ]:
SMOOTH_WINDOW = 2
# For combined plots (both algorithms on same axes)
plot_metrics(reinforce_metrics, ppo_metrics, window=SMOOTH_WINDOW, mode="COMBINED")

In [ ]:
# For separate plots (each algorithm on its own figure)
# plot_metrics(reinforce_metrics, ppo_metrics, window=SMOOTH_WINDOW, mode="SEPARATE")

### Test model saving and testing

In [ ]:
test_metrics = test_saved_model(
    model_file=MODEL_FILE,
    test_data_file=TEST_FILE,
    wear_threshold=WEAR_THRESHOLD,
    r1=R1_CONTINUE,
    r2=R2_REPLACE,
    r3=R3_VIOLATION
)

#### Utility functions

In [ ]:
# import pandas as pd
# from utilities import downsample
# # Read the CSV file

# DATA = 'Case_1_1K_Data'
# DATA_TO_DOWNSAMPLE = f'data\{DATA}.csv'
# DATA_TO_DOWNSAMPLE_DS = f'data\DS_{DATA}.csv'

# df_raw = pd.read_csv(DATA_TO_DOWNSAMPLE)
# df = downsample(df_raw, 20)
# df.to_csv(DATA_TO_DOWNSAMPLE_DS, index=False)